In [108]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

In [109]:
#read csv file
city_data = pd.read_csv("Output_data/cities.csv" ,index_col=0)
city_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Tuktoyaktuk,20,CA,1666460710,92,69.4541,-133.0374,12.20,5.75
1,Hithadhoo,82,MV,1666460747,73,-0.6000,73.0833,81.97,7.70
2,Ribeira Grande,20,PT,1666460734,64,38.5167,-28.7000,69.78,21.85
3,Mana,28,GF,1666460748,62,5.6592,-53.7767,87.33,11.63
4,Pisco,18,PE,1666460749,77,-13.7000,-76.2167,66.02,11.50


# Humidity Heatmap

In [110]:
#configure gmaps with API key
gmaps.configure(api_key=g_key)


In [111]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["Lat", "Lng"]].astype(float)
weights= city_data["Humidity"]

# Create HeatMap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights = weights,
                              dissipating=False, max_intensity=100,
                              point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [120]:
# A max temperature lower than 80 degrees but higher than 70
# Wind speed less than 10 mph
# Zero cloudiness
new_df = city_data.loc[(city_data["Max Temp"]>=70) & (city_data["Max Temp"]<80)]
new_df = new_df.loc[(new_df["Wind Speed"]>10) & (new_df["Cloudiness"]==0)]
hotel_df = new_df.reset_index(drop=True)
# remove rows with null values
hotel_df.dropna()
# New coulmn is added
hotel_df["Hotel Name"] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Cape Town,0,ZA,1666460711,72,-33.9258,18.4232,71.38,16.11,
1,Comodoro Rivadavia,0,AR,1666460758,6,-45.8667,-67.5000,71.46,23.02,
2,Saint George,0,US,1666460527,26,37.1041,-113.5841,78.26,12.66,
3,Karratha,0,AU,1666460832,64,-20.7377,116.8463,73.56,15.50,
4,Port Hedland,0,AU,1666460837,73,-20.3167,118.5667,75.25,13.80,
5,Calama,0,CL,1666460857,5,-22.4667,-68.9333,73.89,20.71,
6,Hamilton,0,US,1666460855,47,39.1834,-84.5333,76.06,12.66,
7,Pimentel,0,PE,1666460873,60,-6.8367,-79.9342,70.09,12.66,
8,Lapeer,0,US,1666460917,25,43.0514,-83.3188,74.25,11.50,
9,Havelock,0,US,1666460945,57,34.8791,-76.9013,76.95,12.66,


In [121]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key}

#looping through the dataframe
for index,row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    
     # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
     # convert to json
    response = hotel_name.json()
    try:
        hotel_df.loc[index,"Hotel Name"]=response['results'][0]['name']        
    except (KeyError, IndexError):
        print("Hotel name not found... skipping.")
#     pprint(response)

In [123]:
# display the dataframe with new values
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Cape Town,0,ZA,1666460711,72,-33.9258,18.4232,71.38,16.11,Grande Kloof Boutique Hotel
1,Comodoro Rivadavia,0,AR,1666460758,6,-45.8667,-67.5000,71.46,23.02,Lucania Palazzo Hotel
2,Saint George,0,US,1666460527,26,37.1041,-113.5841,78.26,12.66,Hilton Garden Inn St. George
3,Karratha,0,AU,1666460832,64,-20.7377,116.8463,73.56,15.50,ibis Styles Karratha
4,Port Hedland,0,AU,1666460837,73,-20.3167,118.5667,75.25,13.80,Hospitality Port Hedland
5,Calama,0,CL,1666460857,5,-22.4667,-68.9333,73.89,20.71,MDS Hotel Calama
6,Hamilton,0,US,1666460855,47,39.1834,-84.5333,76.06,12.66,Graduate Cincinnati
7,Pimentel,0,PE,1666460873,60,-6.8367,-79.9342,70.09,12.66,Hotel Puerto del Sol
8,Lapeer,0,US,1666460917,25,43.0514,-83.3188,74.25,11.50,"Holiday Inn Express Lapeer, an IHG Hotel"
9,Havelock,0,US,1666460945,57,34.8791,-76.9013,76.95,12.66,Hampton Inn Havelock


In [124]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

# Create HeatMap

fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

plt.savefig("Output_data/markers.png")

fig



Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [125]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))